# Sparse View Analysis
Use this notebook to summarize how reconstruction quality trends as you reduce the number of input views.

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

sns.set(context="talk", style="ticks")

EXPERIMENT_PATH = Path("experiments/lego")
VIEWS = ["3_views", "5_views", "10_views", "20_views", "full"]
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Experiments target:", EXPERIMENT_PATH)
print("Views:", VIEWS)


In [ ]:
metrics = {}
for view in VIEWS:
    csv_path = EXPERIMENT_PATH / view / "metrics.csv"
    if csv_path.exists():
        metrics[view] = pd.read_csv(csv_path)
    else:
        print(f"missing metrics for {view}: {csv_path}")

df = pd.concat(
    [metrics[view].assign(views=view) for view in VIEWS if view in metrics],
    ignore_index=True,
)
df["views"] = df["views"].astype(str)
df.sample(5)


In [ ]:
def save_plot(fig, name):
    out_path = RESULTS_DIR / "plots" / name
    out_path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_path, bbox_inches="tight")
    print("saved", out_path)

for metric in ["psnr", "ssim", "lpips"]:
    if metric not in df.columns:
        continue
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.barplot(x="views", y=metric, data=df, ax=ax, palette="viridis")
    ax.set_title(f"{metric.upper()} vs number of views")
    ax.set_xlabel("number of views")
    ax.set_ylabel(metric.upper())
    save_plot(fig, f"{metric}_vs_views.png")
    plt.show()


## Analysis Notes
- Insert short observations about whether PSNR/SSIM grow monotonically as view count increases.
- Highlight the gap between extremes (3 vs full) and call out any anomalous LPIPS/SSIM behavior.
- If you plan to showcase renders, load a few images from `experiments/lego/<subset>/renders/` here.

In [ ]:
SUMMARY_CSV = RESULTS_DIR / "metrics.csv"
df.to_csv(SUMMARY_CSV, index=False)
print("expanded metrics saved to", SUMMARY_CSV)
